# Ejercicio 3 -   Regresión Lineal con Keras

En este ejercicio, tu objetivo será entrenar modelos de Regresión Lineal utilizando Keras (y Tensorflow como backend) para familiarizarte con la librería y comprender la relación de sus clases y métodos con los que definimos en los ejercicios 1 y 2.



In [ ]:

!pip install -q rnutil
import rnutil

%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Creando modelos de regresión y prediciendo valores

El siguiente codigo crea modelos de Keras con distintos valores de `w` y `b` y verifica su salida.

Para crear un modelo de Keras utilizamos la clase `Sequential`, que permite utilizar modelos de varias capas. No obstante, en este caso vamos a crear modelos con una sola capa, la capa de clase `Dense` (también conocida como `fully connected` o `lineal`), que permite hacer regresión lineal con varias variables de entrada y de salida. 

Al crear la capa, especificamos la dimensionalidad de salida (1 en este caso) y la de entrada (2 en este caso). De esta forma el modelo puede crear e inicializar los parámetros `W` y `b`.

Como estamos probando un modelo puramente lineal, especificamos `activation=None` para que Keras no agregue ninguna función no-lineal a la salida.

Por último, utilizamos los parámetros por nombre `kernel_initializer` y `bias_initializer` de la clase `Dense` para especificar como inicializar los parámetros `w` (kernel) y `b` (bias) respectivamente. En este caso, utilizamos `keras.initializers.Constant` para inicializarlo con algunos valores constantes.

Ejecuta el siguiente bloque para verificar que en las 4 pruebas la función `predict` de los modelos de Keras hace lo mismo que la función `forward` vista anteriormente. En cada una de las pruebas, estamos inicializando el modelo con distintos valores de `w` y `b`.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

x=np.array([[1.0,2.0]
            ,[2.0,3.0]
            ,[3.0,4.0]])
dimensionalidad_salida=1
dimensionalidad_entrada=(2,)

# PRUEBA 1
# Defino un modelo con w=(0,0) y b=0
model1 = keras.Sequential([
    keras.layers.Dense(dimensionalidad_salida
                       # dimensionalidad de la entrada
                       ,input_shape=dimensionalidad_entrada
                       # activation=None para que no tenga f de activacion (r lineal)
                       ,activation=None 
                       # inicializo w=(0,0)
                      , kernel_initializer = keras.initializers.Constant(value=0)
                       # inicializo b=0
                      , bias_initializer   = keras.initializers.Constant(value=0))
])
y_prediccion = model1.predict(x)
y=np.zeros((3,1))
rnutil.verificar_igualdad(y,y_prediccion)

  
# PRUEBA 2
# Defino un modelo con w=(1,1) y b=0
model2 = keras.Sequential([
    keras.layers.Dense(dimensionalidad_salida
                       ,input_shape=dimensionalidad_entrada
                       , activation=None
                      , kernel_initializer = keras.initializers.Constant(value=1)
                      , bias_initializer   = keras.initializers.Constant(value=0))
])

y_prediccion = model2.predict(x)
y=np.array([[3.0,5.0,7.0]]).T
rnutil.verificar_igualdad(y,y_prediccion)

# PRUEBA 3
# Defino un modelo con w=(0,0) y b=1
model3 = keras.Sequential([
    keras.layers.Dense(dimensionalidad_salida
                       ,input_shape=dimensionalidad_entrada
                       , activation=None
                      , kernel_initializer = keras.initializers.Constant(value=0)
                      , bias_initializer   = keras.initializers.Constant(value=1))
])
y=np.ones((3,1))
y_prediccion = model3.predict(x)
rnutil.verificar_igualdad(y,y_prediccion)

# PRUEBA 4
# Defino un modelo con w=(1,1) y b=1
model4 = keras.Sequential([
    keras.layers.Dense(dimensionalidad_salida
                       ,input_shape=dimensionalidad_entrada
                       , activation=None
                      , kernel_initializer = keras.initializers.Constant(value=1)
                      , bias_initializer   = keras.initializers.Constant(value=1))
])
y_prediccion = model4.predict(x)
y=np.array([[4.0,6.0,8.0]]).T
rnutil.verificar_igualdad(y,y_prediccion)


# Entrenar un modelo de Regresión Lineal con Keras para el dataset de estudio 2D

El siguiente código carga un dataset de prueba con 2 dimensiones de entrada y una de salida.

Luego crea un modelo de regresión lineal con Keras, y visualiza sus pesos iniciales. 

Es importante notar tres cosas:

1. La métrica utilizada es `'mse'`, es decir el error cuadrático medio o promedio. Esta es la misma métrica vista en la teoría de Regresión Lineal.

2. El optimizador es una clase que define el algoritmo para minimizar el error cuadrático. En general, son todas variantes de descenso de gradiente. En este caso, estamos utilizando descenso de gradiente estocástico (`keras.optimizers.SGD`), que es igual al descenso de gradiente pero realiza cada actualización de los parámetros con un subconjunto de los ejemplos del dataset. 

3. El método para entrenar el modelo es `fit`. En este caso, el parámetro `lr` lo recibe el optimizador, pero `fit` recibe la cantidad de iteraciones (`epochs`) y el tamaño del batch para el SGD (`batch_size`).


Al finalizar el entrenamiento, observá los valores del vector de pesos `w`. ¿A qué atributo o variable de entrada le da más importancia el modelo?

In [ ]:
# Carga del dataset

import matplotlib.pyplot as plt

data = rnutil.load_dataset_numpy("study_regression_2d_small.csv")

x,y=data[:,0:2],data[:,2:3]
n,d_in=x.shape
n,d_out=y.shape

# Creación del modelo inicial
print("Inicialización aleatoria del modelo; vuelve a correr esta celda para obtener otros resultados")
# Creo un modelo lineal
modelo = keras.Sequential([
    keras.layers.Dense(d_out,input_shape=(d_in,), activation=None)])

# visualización del modelo inicial
mensaje=f"Modelo inicial"
w,b=modelo.get_weights()

rnutil.plot_regresion_lineal(w,b,x[:,0],x[:,1],y,x1_label="Horas",x2_label="Promedio",y_label="Nota",title=mensaje)


#Creo el optimizador y compilo el modelo para usarlo
α=0.001
# Algoritmo de optimización: Descenso de Gradiente Estocástico (Stochastic Gradient Descent)
sgd = keras.optimizers.SGD(learning_rate=α)
# error cuadrático medio es la métrica de error a optimizar
error_metric='mse' # IMPORTANTE

modelo.compile(
  optimizer=sgd,
  loss=error_metric,
  metrics=['mae'], # metricas para ir calculando en cada iteracion o batch (ninguna ahora)
)


# Entrenamiento del modelo
history  = modelo.fit(x,y,epochs=15,batch_size=32)

# dibujar curva de error
rnutil.plot_loss(history.history["loss"])


# visualiza el modelo y los datos
w,b=modelo.get_weights()

rnutil.plot_regresion_lineal(w,b,x[:,0],x[:,1],y,x1_label="Horas",x2_label="Promedio",y_label="Nota",title="Modelo Final")



